# Project 3.2

Code can be found here:
https://github.com/arezae4/fair-logloss-classification

based on this paper: https://arxiv.org/pdf/1903.03910.pdf

in-process method that jointly optimizes a fairness transformation and linear feature-based parameters for an exponential family distribution that can be viewed as truncated logistic regression

In [1]:
from abc import ABC,abstractmethod
from enum import Enum
from math import isclose
import numpy as np
from scipy.optimize import fmin_bfgs, minimize
import pdb
__all__ = ['DP_fair_logloss_classifier','EODD_fair_logloss_classifier','EOPP_fair_logloss_classifier']

def _log_logistic(X):
    """ This function is used from scikit-learn source code. Source link below """

    """Compute the log of the logistic function, ``log(1 / (1 + e ** -x))``.
    This implementation is numerically stable because it splits positive and
    negative values::
        -log(1 + exp(-x_i))     if x_i > 0
        x_i - log(1 + exp(x_i)) if x_i <= 0

    Parameters
    ----------
    X: array-like, shape (M, N)
        Argument to the logistic function

    Returns
    -------
    out: array, shape (M, N)
        Log of the logistic function evaluated at every point in x
    Notes
    -----
    Source code at:
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/extmath.py
    -----

    See the blog post describing this implementation:
    http://fa.bianp.net/blog/2013/numerical-optimizers-for-logistic-regression/
    """
    if X.ndim > 1: raise Exception("Array of samples cannot be more than 1-D!")
    out = np.empty_like(X) # same dimensions and data types

    idx = X>0
    out[idx] = -np.log(1.0 + np.exp(-X[idx]))
    out[~idx] = X[~idx] - np.log(1.0 + np.exp(X[~idx]))
    return out

def _dot_intercept(w, X):
    """ This function is used from scikit-learn source code. Source link below """

    """Computes y * np.dot(X, w).
    It takes into consideration if the intercept should be fit or not.
    Parameters
    ----------
    w : ndarray, shape (n_features,) or (n_features + 1,)
        Coefficient vector.
    X : {array-like, sparse matrix}, shape (n_samples, n_features)
        Training data.
    y : ndarray, shape (n_samples,)
        Array of labels.
    Returns
    -------
    w : ndarray, shape (n_features,)
        Coefficient vector without the intercept weight (w[-1]) if the
        intercept should be fit. Unchanged otherwise.
    c : float
        The intercept.
    yz : float
        y * np.dot(X, w).
    
    Notes
	-----
	Source code at:
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/linear_model/logistic.py

    """
    c = 0
    if w.size == X.shape[1] + 1:
        c = w[-1]
        w = w[:-1]

    z = np.dot(X, w) + c
    return z

def sum_truncated_loss_grad(theta, X, Y, idx1, idx0, _lambda ):
    z = _dot_intercept(theta, X)
    p = np.exp(_log_logistic(z))
    loss , grad = 0, 0
    if _lambda == 0:
        loss, grad = sum_logistic_loss_grad(theta,X,Y, np.logical_or(idx1 , idx0))
        cond_g1, cond_g0 = idx1, idx0
    else:
        p1 = np.mean(idx1)   # empirical probability of each group
        p0 = np.mean(idx0)
        if _lambda > 0:
            cond_g1 = np.logical_and( idx1 , p > p1 / _lambda)
            cond_g0 = np.logical_and( idx0 , p < (1 - p0 / _lambda))

            grad = np.sum(X[np.logical_and(cond_g1, 1 - Y)] ,0) + np.sum(-X[np.logical_and(cond_g0, Y)], 0)
            loss = (np.log(_lambda /p1)) * np.sum(cond_g1,0) + np.sum(_dot_intercept(theta,X[cond_g1]) * (1-Y[cond_g1]),0) \
                   + (np.log(_lambda/p0)) * np.sum(cond_g0,0) + np.sum(-_dot_intercept(theta,X[cond_g0]) * Y[cond_g0],0) 
        else:
            cond_g1 = np.logical_and( idx1 , p < (1 + p1 / _lambda))
            cond_g0 = np.logical_and( idx0 , p > - p0 / _lambda)

            grad = np.sum(X[np.logical_and(cond_g0, 1 - Y)],0) + np.sum(-X[np.logical_and(cond_g1, Y)], 0)
            loss = (np.log(-_lambda/p1)) * np.sum(cond_g1,0) + np.sum(-_dot_intercept(theta,X[cond_g1]) * Y[cond_g1],0) \
                   + (np.log(-_lambda/p0)) * np.sum(cond_g0,0) + np.sum(_dot_intercept(theta,X[cond_g0]) * (1-Y[cond_g0]),0) 
    return loss, grad, np.logical_or(cond_g1, cond_g0)             

def sum_logistic_loss_grad(theta, X, Y, idx):
    X, Y = X[idx,:], Y[idx]
    z = _dot_intercept(theta, X)
    p = np.exp(_log_logistic(z))
    grad = np.dot(p.T, X) - np.sum(X[Y == 1,:],0)

    logZ = z + np.log(1 + np.exp(-z))
    loss = np.sum(logZ,0) - np.sum(z * Y,0)
    
    return loss, grad 

def fairify(a,b):
    avgA, avgB = np.mean(a), np.mean(b)
    if a.size == 0:
        return np.Inf 
    elif b.size == 0:
        return np.NINF
    _lambda = 0
    if isclose(avgA,avgB):
        return _lambda  # already fair
    flipped = False
    if avgA < avgB:
        b, a = a, b
        avgA, avgB = avgB, avgA
        flipped = True
    diff = avgA - avgB
    if diff < 0:
        raise ValueError('_lambda is not supposed to be negative')
    
    a = - np.sort(-a)     # sort descending
    b.sort()                 # sort ascending

    idxA, idxB = 0, 0   # current index
    thrA, thrB = 1.0, 0.0   # current probability threshold
    gainA, gainB = 0, 0     # average gain in each group

    while True:
        if idxA < len(a):
            cd_thrA = a[idxA]
        else:
            cd_thrA = 0.0
        cd_thrB_ifA = 1 - (cd_thrA * len(b) / len(a))

        if idxB < len(b):
            cd_thrB = b[idxB]
        else:
            cd_thrB = 1.0
        
        if cd_thrB_ifA <= cd_thrB:
            next_thrA = cd_thrA
            next_thrB = cd_thrB_ifA
        else:
            next_thrA = (1 - cd_thrB) * len(a) / len(b)
            next_thrB = cd_thrB

        next_gainA = gainA + idxA * (thrA - next_thrA) / len(a)
        next_gainB = gainB + idxB * (next_thrB - thrB) / len(b)

        if isclose(next_gainA + next_gainB , diff):
            thrA = next_thrA
            _lambda = len(a) / thrA
            break
        elif next_gainA + next_gainB < diff:
            thrA = next_thrA
            thrB = next_thrB
            if cd_thrB_ifA <= cd_thrB:
                idxA += 1
            else:
                idxB += 1
            gainA = next_gainA
            gainB = next_gainB
        else:
            gain_needed = diff - gainA - gainB
            _lambda = (idxA + idxB) / (idxA * thrA / len(a) + idxB * (1 - thrB) / len(b) - gain_needed)
            break
    thrA = len(a) / _lambda
    thrB = 1 - len(b) / _lambda
    if flipped:
        _lambda = -_lambda
    avgA = np.mean(np.minimum(a, thrA))
    avgB = np.mean(np.maximum(b, thrB))

    if not isclose(avgA , avgB):
        raise ValueError('Averages not equalized %.3f vs %.3f, diff was %.3f' % (avgA, avgB, diff) )
    return _lambda 


class fair_logloss_classifier:
    def __init__(self, tol=1e-6, verbose=True, max_iter=10000, C = .001, random_initialization=False):
        self.tol = tol
        self.verbose = verbose
        self.max_iter = max_iter
        self.C = C
        self.random_start = random_initialization
        self.theta = None
        
    @abstractmethod
    def compute_loss_grad(self, theta,  X, Y):
        pass        
    @abstractmethod
    def _group_protected_attribute(self, Y, A):
        pass

    def fit(self,X,Y,A):
        n = np.size(Y)
        X = np.hstack((X,np.ones((n,1))))
        m = X.shape[1]
        
        self._group_protected_attribute(Y,A)

        if self.random_start:
            theta = np.random.random_sample(m) - .5 
        else:
            theta = np.zeros((m,))
        #f = lambda w : self.compute_loss_grad(w,X, Y)[0]
        #grad = lambda w : self.compute_loss_grad(w,X, Y)[1] 
        def callback(w):
            f, g = self.compute_loss_grad(w,X,Y)
            print("fun_value {:.4f} \t gnorm {:.4f}".format(f,np.linalg.norm(g)))
        #res = fmin_bfgs(f,theta, grad, gtol=self.tol, maxiter=self.max_iter,full_output=False,disp=True, retall=True, callback = callback)
        #res = minimize(self.compute_loss_grad, theta,args=(X, Y), method='L-BFGS-B',jac=True, tol=self.tol, options={'maxiter':self.max_iter, 'disp':False}, callback=callback)
        res = minimize(self.compute_loss_grad, theta,args=(X, Y), method='L-BFGS-B',jac=True, tol=self.tol, options={'maxiter':self.max_iter, 'disp':self.verbose})
         
        self.theta = res.x
        return self

    @abstractmethod
    def predict_proba_given_y(self,X,Y,A):
        pass
    @abstractmethod
    def predict_proba(self,X,A):
        pass

    def _fixed_point_proba(self,X,A):
        phat1, pcheck1 = self.predict_proba_given_y(X,np.ones_like(A),A)
        phat0, pcheck0 = self.predict_proba_given_y(X,np.zeros_like(A),A)
        prob = (phat1 * pcheck0 + phat0 * (1 - pcheck1)) / (1 - pcheck1 + pcheck0)
        return prob
         
    def _predict_proba_given_y(self, X, Y, _lambda, grp1, grp2, p1, p2):
        """
            Computes \hat{P}(\hat{Y}|X,A,Y) for two groups

        Parameters
        ----------
        X : ndarray, shape (n_samples,n_features + 1)
            Feature vector
        Y : ndarray, shape (n_samples,)
            Array of labels.
        _lambda : float 
            Fairness parameter for the given two groups
        grp1, grp2 : ndarray, shape(n_samples,)
            indices for group members
        p1, p2, : float
            empirical probability of each group from training data
        
        Returns
        -------
        phat: array, shape (n_samples,) 
            Prediction probability
        pcheck : array, shape (n_samples,)
            Adversarial estimation of the empirical distribution
        """
        phat = np.exp(_log_logistic(_dot_intercept(self.theta,X)))
        pcheck = np.copy(phat)
        
        if _lambda > 0:
            cond = (p1 / _lambda) < phat
            phat[grp1] = np.minimum( phat[grp1] , p1 /_lambda)
            pcheck[grp1] = np.where( cond[grp1], np.ones_like(pcheck[grp1]), phat[grp1] * (1 + (_lambda / p1) * (1 - phat[grp1])))
            
            cond = (1 - (p2 / _lambda)) > phat
            phat[grp2] = np.maximum( phat[grp2] , 1 - p2 /_lambda)
            pcheck[grp2] = np.where( cond[grp2], np.zeros_like(cond[grp2]), phat[grp2] * (1 - (_lambda / p2) * (1 - phat[grp2])))
        elif _lambda < 0:
            cond = (1 + (p1 / _lambda)) > phat
            phat[grp1] = np.maximum( phat[grp1] , 1 + p1 /_lambda)
            pcheck[grp1] = np.where( cond[grp1], np.zeros_like(cond[grp1]), phat[grp1] * (1 - (_lambda / p1) * (1 - phat[grp1])))
            
            cond = (- p2 / _lambda) < phat
            phat[grp2] = np.minimum( phat[grp2] , - p2 /_lambda)
            pcheck[grp2] = np.where( cond[grp2], np.ones_like(cond[grp2]), phat[grp2] * (1 + (_lambda / p2) * (1 - phat[grp2])))
        return phat, pcheck

    def predict(self,X,A):
        return np.round(self.predict_proba(X,A))

    @abstractmethod
    def fairness_violation(self,X,Y,A):
        pass
    def score(self,X,Y,A):
        return 1 - np.mean(abs(self.predict(X,A) - Y))
    def expected_error(self,X,Y,A):
        proba = self.predict_proba(X,A)
        return np.mean(np.where(Y == 1 , 1 - proba, proba))

class DP_fair_logloss_classifier(fair_logloss_classifier):
    def __init__(self, tol=1e-6, verbose=True, max_iter=10000, C = .1, random_initialization=False):
        super().__init__(tol = tol, verbose = verbose, max_iter = max_iter, C = C, random_initialization=random_initialization)

    def _group_protected_attribute(self, tr_Y, tr_A):
        self.grp1 = tr_A == 1
        self.grp2 = tr_A == 0

    def compute_loss_grad(self, theta, X, Y):
        p = np.exp(_log_logistic(_dot_intercept(theta,X)))
        idx1 = self.grp1 # A == 1
        idx0 = self.grp2 # A == 0
        n = X.shape[0]
        self._lambda = fairify(p[idx1], p[idx0]) / n
        loss, grad, trunc_idx = sum_truncated_loss_grad(theta, X, Y, idx1, idx0, self._lambda)
        loss_ow, grad_ow = sum_logistic_loss_grad(theta,X,Y, np.logical_not(trunc_idx))

        loss += loss_ow
        grad += grad_ow
        
        loss = loss / n + .5 * self.C * np.dot(theta, theta)
        grad = grad / n + self.C * theta
        return loss, grad
    
    def predict_proba_given_y(self,X,Y,A):
        grp1 = A == 1
        grp2 = A == 0
        p1, p2 = np.mean(self.grp1) , np.mean(self.grp2) # group empirical probability based on training data
        
        return self._predict_proba_given_y(X,Y, self._lambda, grp1, grp2, p1, p2)
                   
    def predict_proba(self,X,A):
        return self.predict_proba_given_y(X,np.empty_like(A),A)[0]

    def fairness_violation(self,X,Y,A):
        proba = self.predict_proba(X,A)
        return abs(np.mean(proba[A == 1]) - np.mean(proba[A == 0]))


class EOPP_fair_logloss_classifier(fair_logloss_classifier):
    def __init__(self, tol=1e-6, verbose=True, max_iter=10000, C = .1, random_initialization=False):
        super().__init__(tol = tol, verbose = verbose, max_iter = max_iter, C = C, random_initialization= random_initialization)

    def _group_protected_attribute(self, tr_Y, tr_A):
        self.grp1 = np.logical_and( tr_A == 1, tr_Y == 1)
        self.grp2 = np.logical_and( tr_A == 0, tr_Y == 1)

    def compute_loss_grad(self, theta, X, Y):
        p = np.exp(_log_logistic(_dot_intercept(theta,X)))
        idx1 = self.grp1 # np.logical_and(A == 1, Y == 1)
        idx0 = self.grp2 # np.logical_and(A == 0, Y == 1)
        n = X.shape[0]
        self._lambda = fairify(p[idx1], p[idx0]) / n 
        loss, grad, trunc_idx  = sum_truncated_loss_grad(theta, X, Y, idx1, idx0, self._lambda)
        loss_ow, grad_ow = sum_logistic_loss_grad(theta,X,Y, np.logical_not(trunc_idx))

        loss += loss_ow
        grad += grad_ow
        loss = loss / n + .5 * self.C * np.dot(theta, theta)
        grad = grad /n + self.C * theta
        return loss, grad

    def predict_proba_given_y(self,X,Y,A):
        grp1 = np.logical_and(A == 1, Y == 1)
        grp2 = np.logical_and(A == 0, Y == 1)
        p1, p2 = np.mean(self.grp1) , np.mean(self.grp2)
        return self._predict_proba_given_y(X,Y, self._lambda, grp1, grp2, p1, p2)
        
            
    def predict_proba(self,X,A):
        return self._fixed_point_proba(X,A)

    def fairness_violation(self,X,Y,A):
        proba = self.predict_proba(X,A)
        return abs(np.mean(proba[np.logical_and(Y == 1, A == 1)]) - np.mean(proba[np.logical_and(Y == 1, A == 0)]))  
    

class EODD_fair_logloss_classifier(fair_logloss_classifier):
    def __init__(self, tol=1e-6, verbose=True, max_iter=10000, C = .1, random_initialization=False):
        super().__init__(tol = tol, verbose = verbose, max_iter = max_iter, C = C, random_initialization=random_initialization)

    def _group_protected_attribute(self, tr_Y, tr_A):
        self.grp1 = np.logical_and( tr_A == 1, tr_Y == 1)
        self.grp2 = np.logical_and( tr_A == 0, tr_Y == 1)
        self.grp3 = np.logical_and( tr_A == 1, tr_Y == 0)
        self.grp4 = np.logical_and( tr_A == 0, tr_Y == 0)
         
    def compute_loss_grad(self, theta,X, Y):
        p = np.exp(_log_logistic(_dot_intercept(theta,X)))
        n = X.shape[0]
        idx11 = self.grp1 # np.logical_and(A == 1, Y == 1)
        idx01 = self.grp2 # np.logical_and(A == 0, Y == 1)
        self._lambda1 = fairify(p[idx11], p[idx01]) / n 

        loss1, grad1, trunc_idx1 = sum_truncated_loss_grad(theta, X, Y, idx11, idx01, self._lambda1)

        idx10 = self.grp3 # np.logical_and(A == 1, Y == 0)
        idx00 = self.grp4 # np.logical_and(A == 0, Y == 0)
        self._lambda0 = fairify(p[idx10], p[idx00]) / n 

        loss0, grad0, trunc_idx2 = sum_truncated_loss_grad(theta, X, Y, idx10, idx00, self._lambda0)
        
        loss_ow, grad_ow = sum_logistic_loss_grad(theta, X, Y, np.logical_not(np.logical_or(trunc_idx1, trunc_idx2)))

        loss = loss1 + loss0 + loss_ow
        grad = grad1 + grad0 + grad_ow
        loss = loss / n + .5 * self.C * np.dot(theta, theta)
        grad = grad /n + self.C * theta
        #pdb.set_trace()
        return loss, grad    
    
    def predict_proba_given_y(self,X,Y,A):
        grp1 = np.logical_and(A == 1, Y == 1)
        grp2 = np.logical_and(A == 0, Y == 1)
        p1, p2 = np.mean(self.grp1) , np.mean(self.grp2)
        phat, pcheck = self._predict_proba_given_y(X, Y, self._lambda1, grp1, grp2, p1, p2)
        
        grp3 = np.logical_and(A == 1, Y == 0)
        grp4 = np.logical_and(A == 0, Y == 0)
        p3, p4 = np.mean(self.grp3) , np.mean(self.grp4)
        phat_, pcheck_ = self._predict_proba_given_y(X, Y, self._lambda0, grp3, grp4, p3, p4)
        idx = np.logical_or(grp3, grp4)
        phat[idx], pcheck[idx] = phat_[idx], pcheck_[idx]

        return phat, pcheck
        

    def predict_proba(self,X,A):
        return self._fixed_point_proba(X,A)
        
    def fairness_violation(self,X,Y,A):
        proba = self.predict_proba(X,A)
        return  abs(np.mean(proba[np.logical_and(Y == 1, A == 1)]) - np.mean(proba[np.logical_and(Y == 1, A == 0)]))  \
            +   abs(np.mean(proba[np.logical_and(Y == 0, A == 1)]) - np.mean(proba[np.logical_and(Y == 0, A == 0)]))
   
     
            

In [2]:
import pandas as pd

def prepare_IBM_adult():
    dataA = pd.read_csv('../data/IBM_adult_A.csv',sep='\t',index_col = 0,header=None)#,usecols=range(1,2))
    dataY = pd.read_csv('../data/IBM_adult_Y.csv',sep='\t',index_col = 0,header=None)#,usecols=range(0,2))
    dataX = pd.read_csv('../data/IBM_adult_X.csv',sep='\t',index_col = 0)
    perm = np.genfromtxt('../data/adult_perm.csv', delimiter=',')
    return dataA.iloc[:,0],dataY.iloc[:,0],dataX,perm 


In [3]:
from __future__ import print_function
from scipy.io import arff
# from prepare_data import prepare_compas,prepare_IBM_adult, prepare_law

import functools
import numpy as np
import pandas as pd
import sys
# from fair_logloss import DP_fair_logloss_classifier, EOPP_fair_logloss_classifier, EODD_fair_logloss_classifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression


def compute_error(Yhat,proba,Y):
    err = 1 - np.sum(Yhat == Y) / Y.shape[0] 
    exp_zeroone = np.mean(np.where(Y == 1 , 1 - proba, proba))
    return err, exp_zeroone

def confu_metrics(confusion_m):
    #Confusion Matrix
    TN, FP, FN, TP = confusion_m.ravel()
    
    N = TP+FP+FN+TN #Total population
    ACC = (TP+TN)/N #Accuracy
    TPR = TP/(TP+FN) # True positive rate
    FPR = FP/(FP+TN) # False positive rate
    FNR = FN/(TP+FN) # False negative rate
    PPP = (TP + FP)/N # % predicted as positive
    
    return FNR

def fairness_metrics(y_true,y_pred,A):
    """Calculate fairness for subgroup of population"""
    A.reset_index(inplace=True,drop = True)
    ind_m = A[A==1].index.tolist()
    ind_f = A[A==0].index.tolist()
    
    ind_m = [int(i) for i in ind_m]
    ind_f = [int(i) for i in ind_f]
    
    cm_m = confusion_matrix(y_true[ind_m],y_pred[ind_m])
    cm_f = confusion_matrix(y_true[ind_f],y_pred[ind_f])
    FNR = confu_metrics(confusion_matrix(y_true,y_pred))
    FNR_m = confu_metrics(cm_m)
    FNR_f = confu_metrics(cm_f)
    
    return FNR,FNR_m,FNR_f


if __name__ == '__main__':
    dataA,dataY,dataX,perm = prepare_IBM_adult()
    dataset = "adult"
    C = .005
    criteria = 'dp'#sys.argv[2]
    if criteria == 'dp':
        h = DP_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)
    elif criteria == 'eqopp':
        h = EOPP_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)
    elif criteria == 'eqodd':
        h = EODD_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)    
    else:
        raise ValueError('Invalid second arg')
    filename_tr = "fairll_{}_{:.3f}_{}_tr.csv".format(dataset,C,criteria)
    filename_ts = "fairll_{}_{:.3f}_{}_ts.csv".format(dataset,C,criteria)
    
    # outfile_tr = open(filename_tr,"w")
    # outfile_ts = open(filename_ts,"w")

    for r in range(20):
        order = perm[r,:]
        tr_sz = int(np.floor(.7 * dataX.shape[0]))
        tr_idx = order[:tr_sz]
        ts_idx = order[tr_sz:]
        tr_X = dataX.reindex(tr_idx)
        ts_X = dataX.reindex(ts_idx)
        
        tr_A = dataA.reindex(tr_X.index)
        ts_A = dataA.reindex(ts_X.index)
        tr_Y = dataY.reindex(tr_X.index)
        ts_Y = dataY.reindex(ts_X.index)
        
        # Comment out to not include A in features
        tr_X = pd.concat([tr_X, tr_A], axis=1) 
        ts_X = pd.concat([ts_X, ts_A], axis=1)
        # ---------

        for c in list(tr_X.columns):
            if tr_X[c].min() < 0 or tr_X[c].max() > 1:
                mu = tr_X[c].mean()
                s = tr_X[c].std(ddof=0)
                tr_X.loc[:,c] = (tr_X[c] - mu) / s
                ts_X.loc[:,c] = (ts_X[c] - mu) / s
        
        h.fit(tr_X.values,tr_Y.values,tr_A.values)
        exp_zo_tr = h.expected_error(tr_X.values, tr_Y.values, tr_A.values)
        exp_zo_ts = h.expected_error(ts_X.values, ts_Y.values, ts_A.values)
        err_tr = 1 - h.score(tr_X.values, tr_Y.values, tr_A.values)
        err_ts = 1 - h.score(ts_X.values, ts_Y.values, ts_A.values)
        violation_tr = h.fairness_violation(tr_X.values, tr_Y.values, tr_A.values)
        violation_ts = h.fairness_violation(ts_X.values, ts_Y.values, ts_A.values)
        prediction = h.predict(ts_X.values, ts_A.values)

        # calculate consistency measure 
        # we use sklearn to find the nearest neighbors
        neigh = NearestNeighbors(n_neighbors=11) # the first neighbour is always the data itself, therefore take 11 neighbours
        nbrs = neigh.fit(ts_X.values)
        distances, indices = nbrs.kneighbors(ts_X.values)
        # calculate C
        sum = 0
        for i in range(len(indices)):
            for j in range(11):
                sum += np.abs(int(prediction[i])-int(prediction[indices[i][j]]))
        result = 1 - sum*(1/(10*len(indices)))
        
        total_FNR_fair,m_FNR_fair,f_FNR_fair = fairness_metrics(ts_Y.values,prediction,ts_A)
        
        
        # train logistic regression 
        clf = LogisticRegression(random_state=0).fit(tr_X.values, tr_Y.values)
        logistic_prediction = clf.predict(ts_X.values)
        # calculate C
        sum = 0
        for i in range(len(indices)):
            for j in range(11):
                sum += np.abs(int(logistic_prediction[i])-int(logistic_prediction[indices[i][j]]))
        result_logistic_regression = 1 - sum*(1/(10*len(indices)))
        
        total_FNR,m_FNR,f_FNR = fairness_metrics(ts_Y.values,logistic_prediction,ts_A)
        
        print("---------------------------- Random Split %d ----------------------------------" % (r + 1))
        print("Train - predict_err : {:.3f} \t expected_err : {:.3f} \t fair_violation : {:.3f} ".format(err_tr, exp_zo_tr,violation_tr))
        print("Test  - predict_err : {:.3f} \t expected_err : {:.3f} \t fair_violation : {:.3f} ".format(err_ts, exp_zo_ts,violation_ts))
        print("accuracy:", accuracy_score(ts_Y.values, prediction))
        print("C =",result)
        print("False Negative Rate total fair: {}, for males: {}, and for females: {}".format(total_FNR_fair,m_FNR_fair,f_FNR_fair))
        print("for non-fair logistic regression:")
        print("accuracy:", accuracy_score(ts_Y.values, logistic_prediction))
        print("C =",result_logistic_regression)
        print("False Negative Rate total: {}, for males: {}, and for females: {}".format(total_FNR,m_FNR,f_FNR))
        print("")

    #     outfile_ts.write("{:.4f},{:.4f},{:.4f}\n".format(exp_zo_ts,err_ts, violation_ts))
    #     outfile_tr.write("{:.4f},{:.4f},{:.4f}\n".format(exp_zo_tr,err_tr, violation_tr))
        
    # outfile_tr.close()
    # outfile_ts.close()

C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 1 ----------------------------------
Train - predict_err : 0.154 	 expected_err : 0.282 	 fair_violation : 0.000 
Test  - predict_err : 0.159 	 expected_err : 0.284 	 fair_violation : 0.006 
accuracy: 0.84145352694037
C = 0.9465246554138719
False Negative Rate total fair: 0.4482453553524034, for males: 0.44459738472126636, and for females: 0.47010309278350515
for non-fair logistic regression:
accuracy: 0.8438858996093462
C = 0.9390285250976634
False Negative Rate total: 0.4093187850191684, for males: 0.39298004129387476, and for females: 0.5072164948453608



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 2 ----------------------------------
Train - predict_err : 0.154 	 expected_err : 0.289 	 fair_violation : 0.000 
Test  - predict_err : 0.152 	 expected_err : 0.288 	 fair_violation : 0.002 
accuracy: 0.8478661457949436
C = 0.9475418294390802
False Negative Rate total fair: 0.4263943919536727, for males: 0.4208096590909091, and for females: 0.46021505376344085
for non-fair logistic regression:
accuracy: 0.850372226726616
C = 0.9393896955848751
False Negative Rate total: 0.3940871685461749, for males: 0.3817471590909091, and for females: 0.46881720430107526



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

---------------------------- Random Split 18 ----------------------------------
Train - predict_err : 0.156 	 expected_err : 0.286 	 fair_violation : 0.000 
Test  - predict_err : 0.152 	 expected_err : 0.284 	 fair_violation : 0.002 
accuracy: 0.8477924375322473
C = 0.9501732144173362
False Negative Rate total fair: 0.42748091603053434, for males: 0.42046641141663765, and for females: 0.4652908067542214
for non-fair logistic regression:
accuracy: 0.8506670597774011
C = 0.9424191051816909
False Negative Rate total: 0.399588960657663, for males: 0.3825269752871563, and for females: 0.4915572232645403



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 19 ----------------------------------
Train - predict_err : 0.156 	 expected_err : 0.285 	 fair_violation : 0.000 
Test  - predict_err : 0.153 	 expected_err : 0.285 	 fair_violation : 0.001 
accuracy: 0.8466868135918036
C = 0.9508513304341417
False Negative Rate total fair: 0.44424620874219445, for males: 0.4397778549114891, and for females: 0.470954356846473
for non-fair logistic regression:
accuracy: 0.8487506449472986
C = 0.9401783739957249
False Negative Rate total: 0.39250669045495096, for males: 0.383547379382159, and for females: 0.4460580912863071



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 20 ----------------------------------
Train - predict_err : 0.153 	 expected_err : 0.282 	 fair_violation : 0.000 
Test  - predict_err : 0.163 	 expected_err : 0.287 	 fair_violation : 0.005 
accuracy: 0.8365887816024177
C = 0.9445566447998821
False Negative Rate total fair: 0.44738389182833627, for males: 0.446873932353946, and for females: 0.45052631578947366
for non-fair logistic regression:
accuracy: 0.842927692194295
C = 0.9353136286577726
False Negative Rate total: 0.40652557319223986, for males: 0.3969935087119918, and for females: 0.4652631578947368



C increases by using the fair logistic regression version.

In [4]:
from __future__ import print_function
from scipy.io import arff
# from prepare_data import prepare_compas,prepare_IBM_adult, prepare_law

import functools
import numpy as np
import pandas as pd
import sys
# from fair_logloss import DP_fair_logloss_classifier, EOPP_fair_logloss_classifier, EODD_fair_logloss_classifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression


def compute_error(Yhat,proba,Y):
    err = 1 - np.sum(Yhat == Y) / Y.shape[0] 
    exp_zeroone = np.mean(np.where(Y == 1 , 1 - proba, proba))
    return err, exp_zeroone

if __name__ == '__main__':
    dataA,dataY,dataX,perm = prepare_IBM_adult()
    dataset = "adult"
    C = .005
    criteria = 'eqopp'#sys.argv[2]
    if criteria == 'dp':
        h = DP_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)
    elif criteria == 'eqopp':
        h = EOPP_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)
    elif criteria == 'eqodd':
        h = EODD_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)    
    else:
        raise ValueError('Invalid second arg')
    filename_tr = "fairll_{}_{:.3f}_{}_tr.csv".format(dataset,C,criteria)
    filename_ts = "fairll_{}_{:.3f}_{}_ts.csv".format(dataset,C,criteria)
    
    # outfile_tr = open(filename_tr,"w")
    # outfile_ts = open(filename_ts,"w")

    for r in range(20):
        order = perm[r,:]
        tr_sz = int(np.floor(.7 * dataX.shape[0]))
        tr_idx = order[:tr_sz]
        ts_idx = order[tr_sz:]
        tr_X = dataX.reindex(tr_idx)
        ts_X = dataX.reindex(ts_idx)
        
        tr_A = dataA.reindex(tr_X.index)
        ts_A = dataA.reindex(ts_X.index)
        tr_Y = dataY.reindex(tr_X.index)
        ts_Y = dataY.reindex(ts_X.index)
        
        # Comment out to not include A in features
        tr_X = pd.concat([tr_X, tr_A], axis=1) 	
        ts_X = pd.concat([ts_X, ts_A], axis=1)
        # ---------

        for c in list(tr_X.columns):
            if tr_X[c].min() < 0 or tr_X[c].max() > 1:
                mu = tr_X[c].mean()
                s = tr_X[c].std(ddof=0)
                tr_X.loc[:,c] = (tr_X[c] - mu) / s
                ts_X.loc[:,c] = (ts_X[c] - mu) / s
        
        h.fit(tr_X.values,tr_Y.values,tr_A.values)
        exp_zo_tr = h.expected_error(tr_X.values, tr_Y.values, tr_A.values)
        exp_zo_ts = h.expected_error(ts_X.values, ts_Y.values, ts_A.values)
        err_tr = 1 - h.score(tr_X.values, tr_Y.values, tr_A.values)
        err_ts = 1 - h.score(ts_X.values, ts_Y.values, ts_A.values)
        violation_tr = h.fairness_violation(tr_X.values, tr_Y.values, tr_A.values)
        violation_ts = h.fairness_violation(ts_X.values, ts_Y.values, ts_A.values)
        prediction = h.predict(ts_X.values, ts_A.values)

        # calculate consistency measure 
        # we use sklearn to find the nearest neighbors
        neigh = NearestNeighbors(n_neighbors=11) # the first neighbour is always the data itself, therefore take 11 neighbours
        nbrs = neigh.fit(ts_X.values)
        distances, indices = nbrs.kneighbors(ts_X.values)
        # calculate C
        sum = 0
        for i in range(len(indices)):
            for j in range(11):
                sum += np.abs(int(prediction[i])-int(prediction[indices[i][j]]))
        result = 1 - sum*(1/(10*len(indices)))
        
        total_FNR_fair,m_FNR_fair,f_FNR_fair = fairness_metrics(ts_Y.values,prediction,ts_A)
        # train logistic regression 
        clf = LogisticRegression(random_state=0).fit(tr_X.values, tr_Y.values)
        logistic_prediction = clf.predict(ts_X.values)
        # calculate C
        sum = 0
        for i in range(len(indices)):
            for j in range(11):
                sum += np.abs(int(logistic_prediction[i])-int(logistic_prediction[indices[i][j]]))
        result_logistic_regression = 1 - sum*(1/(10*len(indices)))
        
        total_FNR,m_FNR,f_FNR = fairness_metrics(ts_Y.values,logistic_prediction,ts_A)
        
        print("---------------------------- Random Split %d ----------------------------------" % (r + 1))
        print("Train - predict_err : {:.3f} \t expected_err : {:.3f} \t fair_violation : {:.3f} ".format(err_tr, exp_zo_tr,violation_tr))
        print("Test  - predict_err : {:.3f} \t expected_err : {:.3f} \t fair_violation : {:.3f} ".format(err_ts, exp_zo_ts,violation_ts))
        print("accuracy:", accuracy_score(ts_Y.values, prediction))
        print("C =",result)
        print("False Negative Rate total fair: {}, for males: {}, and for females: {}".format(total_FNR_fair,m_FNR_fair,f_FNR_fair))
        print("for non-fair logistic regression:")
        print("accuracy:", accuracy_score(ts_Y.values, logistic_prediction))
        print("C =",result_logistic_regression)
        print("False Negative Rate total: {}, for males: {}, and for females: {}".format(total_FNR,m_FNR,f_FNR))
        print("")

    #     outfile_ts.write("{:.4f},{:.4f},{:.4f}\n".format(exp_zo_ts,err_ts, violation_ts))
    #     outfile_tr.write("{:.4f},{:.4f},{:.4f}\n".format(exp_zo_tr,err_tr, violation_tr))
        
    # outfile_tr.close()
    # outfile_ts.close()

C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 1 ----------------------------------
Train - predict_err : 0.155 	 expected_err : 0.231 	 fair_violation : 0.063 
Test  - predict_err : 0.157 	 expected_err : 0.233 	 fair_violation : 0.070 
accuracy: 0.8426328591435099
C = 0.9512862091840495
False Negative Rate total fair: 0.4473606605721026, for males: 0.42980041293874743, and for females: 0.5525773195876289
for non-fair logistic regression:
accuracy: 0.8438858996093462
C = 0.9390285250976634
False Negative Rate total: 0.4093187850191684, for males: 0.39298004129387476, and for females: 0.5072164948453608



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 2 ----------------------------------
Train - predict_err : 0.156 	 expected_err : 0.233 	 fair_violation : 0.061 
Test  - predict_err : 0.153 	 expected_err : 0.231 	 fair_violation : 0.058 
accuracy: 0.8472764796933736
C = 0.949215007002285
False Negative Rate total fair: 0.4221274001828711, for males: 0.40767045454545453, and for females: 0.5096774193548387
for non-fair logistic regression:
accuracy: 0.850372226726616
C = 0.9393896955848751
False Negative Rate total: 0.3940871685461749, for males: 0.3817471590909091, and for females: 0.46881720430107526



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

---------------------------- Random Split 18 ----------------------------------
Train - predict_err : 0.158 	 expected_err : 0.232 	 fair_violation : 0.066 
Test  - predict_err : 0.153 	 expected_err : 0.231 	 fair_violation : 0.064 
accuracy: 0.8472764796933736
C = 0.9528414535269404
False Negative Rate total fair: 0.4459776864357017, for males: 0.4295161851722938, and for females: 0.5347091932457786
for non-fair logistic regression:
accuracy: 0.8506670597774011
C = 0.9424191051816909
False Negative Rate total: 0.399588960657663, for males: 0.3825269752871563, and for females: 0.4915572232645403



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 19 ----------------------------------
Train - predict_err : 0.156 	 expected_err : 0.233 	 fair_violation : 0.062 
Test  - predict_err : 0.153 	 expected_err : 0.232 	 fair_violation : 0.050 
accuracy: 0.8473501879560699
C = 0.9520527751160905
False Negative Rate total fair: 0.42224204579244723, for males: 0.41096841374522736, and for females: 0.4896265560165975
for non-fair logistic regression:
accuracy: 0.8487506449472986
C = 0.9401783739957249
False Negative Rate total: 0.39250669045495096, for males: 0.383547379382159, and for females: 0.4460580912863071



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 20 ----------------------------------
Train - predict_err : 0.152 	 expected_err : 0.231 	 fair_violation : 0.064 
Test  - predict_err : 0.163 	 expected_err : 0.236 	 fair_violation : 0.041 
accuracy: 0.8371047394412914
C = 0.9467089260706125
False Negative Rate total fair: 0.43944738389182836, for males: 0.42910830201571576, and for females: 0.5031578947368421
for non-fair logistic regression:
accuracy: 0.842927692194295
C = 0.9353136286577726
False Negative Rate total: 0.40652557319223986, for males: 0.3969935087119918, and for females: 0.4652631578947368



In [5]:
from __future__ import print_function
from scipy.io import arff
# from prepare_data import prepare_compas,prepare_IBM_adult, prepare_law

import functools
import numpy as np
import pandas as pd
import sys
# from fair_logloss import DP_fair_logloss_classifier, EOPP_fair_logloss_classifier, EODD_fair_logloss_classifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression


def compute_error(Yhat,proba,Y):
    err = 1 - np.sum(Yhat == Y) / Y.shape[0] 
    exp_zeroone = np.mean(np.where(Y == 1 , 1 - proba, proba))
    return err, exp_zeroone

if __name__ == '__main__':
    dataA,dataY,dataX,perm = prepare_IBM_adult()
    dataset = "adult"
    C = .005
    criteria = 'eqodd'#sys.argv[2]
    if criteria == 'dp':
        h = DP_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)
    elif criteria == 'eqopp':
        h = EOPP_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)
    elif criteria == 'eqodd':
        h = EODD_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)    
    else:
        raise ValueError('Invalid second arg')
    filename_tr = "fairll_{}_{:.3f}_{}_tr.csv".format(dataset,C,criteria)
    filename_ts = "fairll_{}_{:.3f}_{}_ts.csv".format(dataset,C,criteria)
    
    # outfile_tr = open(filename_tr,"w")
    # outfile_ts = open(filename_ts,"w")

    for r in range(20):
        order = perm[r,:]
        tr_sz = int(np.floor(.7 * dataX.shape[0]))
        tr_idx = order[:tr_sz]
        ts_idx = order[tr_sz:]
        tr_X = dataX.reindex(tr_idx)
        ts_X = dataX.reindex(ts_idx)
        
        tr_A = dataA.reindex(tr_X.index)
        ts_A = dataA.reindex(ts_X.index)
        tr_Y = dataY.reindex(tr_X.index)
        ts_Y = dataY.reindex(ts_X.index)
        
        # Comment out to not include A in features
        tr_X = pd.concat([tr_X, tr_A], axis=1) 	
        ts_X = pd.concat([ts_X, ts_A], axis=1)
        # ---------

        for c in list(tr_X.columns):
            if tr_X[c].min() < 0 or tr_X[c].max() > 1:
                mu = tr_X[c].mean()
                s = tr_X[c].std(ddof=0)
                tr_X.loc[:,c] = (tr_X[c] - mu) / s
                ts_X.loc[:,c] = (ts_X[c] - mu) / s
        
        h.fit(tr_X.values,tr_Y.values,tr_A.values)
        exp_zo_tr = h.expected_error(tr_X.values, tr_Y.values, tr_A.values)
        exp_zo_ts = h.expected_error(ts_X.values, ts_Y.values, ts_A.values)
        err_tr = 1 - h.score(tr_X.values, tr_Y.values, tr_A.values)
        err_ts = 1 - h.score(ts_X.values, ts_Y.values, ts_A.values)
        violation_tr = h.fairness_violation(tr_X.values, tr_Y.values, tr_A.values)
        violation_ts = h.fairness_violation(ts_X.values, ts_Y.values, ts_A.values)
        prediction = h.predict(ts_X.values, ts_A.values)

        # calculate consistency measure 
        # we use sklearn to find the nearest neighbors
        neigh = NearestNeighbors(n_neighbors=11) # the first neighbour is always the data itself, therefore take 11 neighbours
        nbrs = neigh.fit(ts_X.values)
        distances, indices = nbrs.kneighbors(ts_X.values)
        # calculate C
        sum = 0
        for i in range(len(indices)):
            for j in range(11):
                sum += np.abs(int(prediction[i])-int(prediction[indices[i][j]]))
        result = 1 - sum*(1/(10*len(indices)))
        
        total_FNR_fair,m_FNR_fair,f_FNR_fair = fairness_metrics(ts_Y.values,prediction,ts_A)
        # train logistic regression 
        clf = LogisticRegression(random_state=0).fit(tr_X.values, tr_Y.values)
        logistic_prediction = clf.predict(ts_X.values)
        # calculate C
        sum = 0
        for i in range(len(indices)):
            for j in range(11):
                sum += np.abs(int(logistic_prediction[i])-int(logistic_prediction[indices[i][j]]))
        result_logistic_regression = 1 - sum*(1/(10*len(indices)))
        
        total_FNR,m_FNR,f_FNR = fairness_metrics(ts_Y.values,logistic_prediction,ts_A)
        
        print("---------------------------- Random Split %d ----------------------------------" % (r + 1))
        print("Train - predict_err : {:.3f} \t expected_err : {:.3f} \t fair_violation : {:.3f} ".format(err_tr, exp_zo_tr,violation_tr))
        print("Test  - predict_err : {:.3f} \t expected_err : {:.3f} \t fair_violation : {:.3f} ".format(err_ts, exp_zo_ts,violation_ts))
        print("accuracy:", accuracy_score(ts_Y.values, prediction))
        print("C =",result)
        print("False Negative Rate total Fair: {}, for males: {}, and for females: {}".format(total_FNR_fair,m_FNR_fair,f_FNR_fair))
        print("for non-fair logistic regression:")
        print("accuracy:", accuracy_score(ts_Y.values, logistic_prediction))
        print("C =",result_logistic_regression)
        print("False Negative Rate total: {}, for males: {}, and for females: {}".format(total_FNR,m_FNR,f_FNR))
        print("")

    #     outfile_ts.write("{:.4f},{:.4f},{:.4f}\n".format(exp_zo_ts,err_ts, violation_ts))
    #     outfile_tr.write("{:.4f},{:.4f},{:.4f}\n".format(exp_zo_tr,err_tr, violation_tr))
        
    # outfile_tr.close()
    # outfile_ts.close()

C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 1 ----------------------------------
Train - predict_err : 0.154 	 expected_err : 0.252 	 fair_violation : 0.012 
Test  - predict_err : 0.159 	 expected_err : 0.253 	 fair_violation : 0.025 
accuracy: 0.8410849856268888
C = 0.9479398540576398
False Negative Rate total Fair: 0.4464759657918018, for males: 0.43840330350997936, and for females: 0.4948453608247423
for non-fair logistic regression:
accuracy: 0.8438858996093462
C = 0.9390285250976634
False Negative Rate total: 0.4093187850191684, for males: 0.39298004129387476, and for females: 0.5072164948453608



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 2 ----------------------------------
Train - predict_err : 0.155 	 expected_err : 0.258 	 fair_violation : 0.020 
Test  - predict_err : 0.154 	 expected_err : 0.257 	 fair_violation : 0.015 
accuracy: 0.8464656888037149
C = 0.9477113584432815
False Negative Rate total Fair: 0.44132886315147823, for males: 0.4367897727272727, and for females: 0.46881720430107526
for non-fair logistic regression:
accuracy: 0.850372226726616
C = 0.9393896955848751
False Negative Rate total: 0.3940871685461749, for males: 0.3817471590909091, and for females: 0.46881720430107526



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

---------------------------- Random Split 18 ----------------------------------
Train - predict_err : 0.158 	 expected_err : 0.256 	 fair_violation : 0.022 
Test  - predict_err : 0.154 	 expected_err : 0.254 	 fair_violation : 0.016 
accuracy: 0.8463182722783223
C = 0.9466499594604555
False Negative Rate total Fair: 0.43305930710510865, for males: 0.4291681169509224, and for females: 0.4540337711069418
for non-fair logistic regression:
accuracy: 0.8506670597774011
C = 0.9424191051816909
False Negative Rate total: 0.399588960657663, for males: 0.3825269752871563, and for females: 0.4915572232645403



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 19 ----------------------------------
Train - predict_err : 0.156 	 expected_err : 0.263 	 fair_violation : 0.033 
Test  - predict_err : 0.152 	 expected_err : 0.263 	 fair_violation : 0.021 
accuracy: 0.8476450210068549
C = 0.9496793690572713
False Negative Rate total Fair: 0.42521558132619686, for males: 0.41895175286358904, and for females: 0.46265560165975106
for non-fair logistic regression:
accuracy: 0.8487506449472986
C = 0.9401783739957249
False Negative Rate total: 0.39250669045495096, for males: 0.383547379382159, and for females: 0.4460580912863071



C:\Users\srodriguezb\Anaconda3\envs\introds\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------- Random Split 20 ----------------------------------
Train - predict_err : 0.151 	 expected_err : 0.255 	 fair_violation : 0.029 
Test  - predict_err : 0.163 	 expected_err : 0.260 	 fair_violation : 0.009 
accuracy: 0.8365150733397214
C = 0.9431488169823837
False Negative Rate total Fair: 0.44238683127572015, for males: 0.43969935087119916, and for females: 0.4589473684210526
for non-fair logistic regression:
accuracy: 0.842927692194295
C = 0.9353136286577726
False Negative Rate total: 0.40652557319223986, for males: 0.3969935087119918, and for females: 0.4652631578947368

